In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "tests/test_custom/config_app/"

In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

In [3]:
from etl_pipeline.config import columns_namespace as cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [6]:
from pipelines.ms.wm_address_pipeline import MSPipeline
from config import pipeline_config


In [7]:
# payload_json = {'alertedParty': {'supplementalInfo': {'alertId': 'SANC-ASM-6434404',
#    'uniqueCustId': 'R_US_Active_Address_A00170086018_2018-08-28-18.45.50.012201',
#    'busDate': '20211127',
#    'datasetName': 'R_US_Active_Address',
#    'relatedParties': {'party': [{'id': '0002712291',
#       'fields': {'taxId': '154421273',
#        'taxIdType': 'SSN',
#        'partyResidenceCountry': 'United States',
#        'partyName': 'Pladimir Vutin',
#        'partyLastName': 'Vutin',
#        'partyType': 'Individual',
#        'countryOfIncorporation': None,
#        'dobDate': '02/31/1900',
#        'partyPrimaryCitizenshipCountry': 'United States',
#        'partyCountryOfBirth': None,
#        'partyFirstName': 'Pladimir',
#        'partyMiddleName': None,
#        'partyId': '0002712291'}},
#      {'id': '0004727489',
#       'fields': {'taxId': '022368917',
#        'taxIdType': 'SSN',
#        'partyResidenceCountry': 'United States',
#        'partyName': 'Eva Pladimirova',
#        'partyLastName': 'Pladimirova',
#        'partyType': 'Individual',
#        'countryOfIncorporation': None,
#        'dobDate': '04/31/1910',
#        'partyPrimaryCitizenshipCountry': 'United States',
#        'partyCountryOfBirth': None,
#        'partyFirstName': 'Eva',
#        'partyMiddleName': None,
#        'partyId': '0004727489'}}]},
#    'relatedAccounts': {'account': [{'id': '2021-01-29-13.44.57.093694',
#       'fields': {'branchAccountNumber': '234_142529',
#        'lastName': 'Vutin',
#        'srcSystemAccountKey': '2021-01-29-13.44.57.093694',
#        'firstName': 'Pladimir',
#        'beneficiaryName': 'Pladimir Vutin'}},
#      {'id': '2021-01-29-13.44.57.093694',
#       'fields': {'branchAccountNumber': '234_142529',
#        'lastName': 'Pladimirova',
#        'srcSystemAccountKey': '2021-01-29-13.44.57.093694',
#        'firstName': 'Eva',
#        'beneficiaryName': 'Eva Pladimirova'}}]}},
#   'headerInfo': {'uniqueCustomerId': 'R_RUS_Active_Address_2010-02-11-11.11',
#    'datasetId': '1044',
#    'datasetName': 'R_US_Active_Address',
#    'masterId': '637451852',
#    'currentVersionId': '678175153',
#    'stopDescriptors': {'stopDescriptor': [{'name': 'JOHN DOE'},
#      {'name': 'DOE JOHN'}]},
#    'firstVersionCreatedDt': '2010-02-02T22: 24: 24.222+02: 20',
#    'lastVersionUpdatedDt': '2010-02-02T22: 24: 24.222+02: 20',
#    'masterVersion': 'a172863871263234234',
#    'inputVersionSample': 'R_RUS_Active_Address_2010-02-11-11.11'},
#   'inputRecordHist': {'inputRecords': [{'version': 'a1231231231241343454251234234',
#      'createdDate': '01/05/10',
#      'status': 'X',
#      'masterId': '1231231231241',
#      'versionId': '123',
#      'uniqueCustId': 'R_RUS_Active_Address_2010-02-11-11.11',
#      'fields': [{'name': 'SOURCE_REF',
#        'isScreenable': 'false',
#        'value': 'R_RUS_Active_Address_2010-02-11-11.11',
#        'sortOrder': '1'},
#       {'name': 'UNIQUE_KEY',
#        'isScreenable': 'false',
#        'value': 'ALA12321312305004514758',
#        'sortOrder': '12'},
#       {'name': 'ADDRESS1_COUNTRY',
#        'isScreenable': 'false',
#        'value': 'USA',
#        'sortOrder': '12'}]}]}},
#  'watchlistParty': {'matchRecords': [{'masterId': '63745112123123852',
#     'accountSeq': '12',
#     'datasetId': '1233',
#     'uniqueCustomerId': 'R_RUS_Active_Address_2010-02-11-11.11',
#     'masterVersion': 'a123123123123123123123',
#     'matchId': '1',
#     'matchStatus': 'O',
#     'riskScore': '-2.0',
#     'inputVersionId': '123',
#     'matchType': 'XDC',
#     'entityId': '123123123123',
#     'entityVersion': '1231231423546356456345365',
#     'entity': {'id': '2134',
#      'version': '1215342524314143',
#      'name': 'JOHNNY DOE',
#      'listId': '213123',
#      'listCode': 'EXE',
#      'entityType': '0123',
#      'createdDate': '10/02/1993',
#      'lastUpdateDate': '02/02/2004',
#      'source': 'RBS',
#      'programs': {'program': {'type': 'US',
#        'S8_extracted_value': 'UNITED STANES'}},
#      'addresses': {'address': {'city': 'CHwm_address_in_payload_formatICAGO',
#        'country': 'US',
#        'countryName': 'UNITED STATES OF AMERICA'}}},
#     'entityType': '012',
#     'entityTextType': 'UPISD',
#     'sourceCode': 'RABS',
#     'stopDescriptors': [{'name': 'JOHNNY DOE',
#       'totalMatchScore': '0.33',
#       'stopDescriptorDetails': [{'inputToken': 'hense',
#         'inputSynonym': 'null',
#         'sdToken': 'denim',
#         'matchScore': '0.123'}]}],
#     'firstMatchedDate': '01/05/10',
#     'lastMatchedDate': '01/05/10',
#     'lastReviewDate': '01/05/10'},
#    {'masterId': '12312312432442524',
#     'accountSeq': '12',
#     'datasetId': '1111',
#     'uniqueCustomerId': 'R_RUS_Active_Address_2010-02-11-11.11',
#     'masterVersion': 'asd123123343124141',
#     'matchId': '2',
#     'matchStatus': 'O',
#     'riskScore': '-3.0',
#     'inputVersionId': '123',
#     'matchType': 'SSD',
#     'entityId': '1232123',
#     'entityVersion': '123123231213432423',
#     'entity': {'id': '12321',
#      'version': '32131312313213',
#      'name': 'DOE DOE JOHN',
#      'listId': '123',
#      'listCode': 'EUS',wm_address_in_payload_format
#      'entityType': '02',
#      'createdDate': '12/09/2010',
#      'lastUpdateDate': '08/31/2010',
#      'source': 'EUS',
#      'dobs': {'dob': {'D': '11',
#        'M': '01',
#        'Y': '1924',
#        'S8_extracted_value': '01/11/1924'}},
#      'nationalities': {'nationality': {'ctryCode': 'US',
#        'S8_extracted_value': 'CHIC'}},
#      'aliases': {'alias': {'type': 'Alias',
#        'S8_extracted_value': 'DONNY JOHN'}},
#      'programs': {'program': {'type': 'PPK',
#        'S8_extracted_value': 'UNITED STATES OF AMERICA'}},
#      'titles': {'title': 'PRESIDENT OF THE ACADEMY'},
#      'sdfs': {'sdf': ['09/01/2010', '2010/1209 (OJ L333)', '123143414']},
#      'otherIds': {'childId': '124134314'}},
#     'entityType': '04',
#     'entityTextType': 'UPIDDDD',
#     'sourceCode': 'EUS',
#     'stopDescriptors': [{'name': 'DOE DOE DOE',
#       'totalMatchScore': '0.22',
#       'stopDescriptorDetails': [{'inputToken': 'johnny',
#         'inputSynonym': 'syn',
#         'sdToken': 'doe',
#         'matchScore': '0.223'}]},
#      {'name': 'JOHN JOHN DOE DOE DOE',
#       'totalMatchScore': '0.9999999999999',
#       'stopDescriptorDetails': [{'inputToken': 'doe',
#         'inputSynonym': 'null',
#         'sdToken': 'doe',
#         'matchScore': '3.0'}]}],
#     'firstMatchedDate': '01/05/20',
#     'lastMatchedDate': '01/05/20',
#     'lastReviewDate': '01/05/20'}]}}

In [35]:
with open(f'notebooks/sample/wm_party_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [36]:
payload = payload_json

In [37]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

## transform standardized to cleansed

In [38]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [39]:
from etl_pipeline.config import alert_agents_config

In [40]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [41]:
from etl_pipeline.custom.ms.datatypes.field import InputRecordField

### payload

In [42]:
len(new_payloads)

3

In [43]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
123
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981', '//']
['UNITED STATES OF AMERICA'] ['US']
PAYLOAD
122438659
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981', '//']
['UNITED STATES OF AMERICA'] ['US']
PAYLOAD
122438659
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981', '//']
['UNITED STATES OF AMERICA'] ['PL']


In [44]:
import pickle
with open("tests/shared/parsed_payload_2_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [19]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [20]:
return_key(payload, "")

TypeError: 'InputRecordField' object is not iterable

In [ ]:
with open(f'notebooks/sample/alert_in_payload_format.json', 'w') as file:
    json.dump(new_dict ,file)

In [ ]:
payload_json = PayloadLoader().load_payload_from_json(new_dict)

In [ ]:
payload_json == payload

In [ ]:
with open(f'notebooks/sample/alert.json', 'r') as file:
    payload = json.loads(file.read())

In [ ]:
payload